<a href="https://colab.research.google.com/github/burakerdal66/Makine_Ogrenmesi_Projeleri/blob/main/FilmOnerisi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

İçerik öneri sistem türleri:


1.   İçerik tabanlı öneri
2.   Popülerliğe dayalı öner
3.   İşbirlikçi öneri








*   Veriler toplanacak.
*   Verilen Önişlemeye alınacak( eksik veriler temizlenecek.  vs)
* Özellik çıkarımı yapılacak( metinsel veriler sayısal verilere çevirilecek vs)





Bağımlılıkların içe aktarılması

In [3]:
import numpy as np # numpy dizileri oluşturaağız.
import pandas as pd # csv verilerinden pandas dataframeler oluşturmak için kullanılır.
import difflib #yazımlardaki düzeni ayarlamaya yarar.
from sklearn.feature_extraction.text import TfidfVectorizer # metinleri sayısala çevirip spam tahmin sistemi
from sklearn.metrics.pairwise import cosine_similarity #benzerlik yüzdesi oluştururken kullanacağız.


Verilerin toplanması ve Ön işleme

In [4]:
#cvs dosyasını pandas dataframe ine çevirmek
film_verileri=pd.read_csv("/content/drive/MyDrive/datasetler/movies.csv")

In [5]:
#Veri setinin ilk 5 sütununu verme
film_verileri.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [6]:
#veri çerçevesindeki satır ve sütün sayısı
film_verileri.shape

(4803, 24)

In [7]:
# ilgilendiğimiz  özellikteki sütunların seçilmesi
# film türü, film sloganı,oyuncu kadrosu, yönetmenler ve anahtar kelimeleri alacağız
secilen_ozellikler=["genres","keywords","tagline","cast","director"]
print(secilen_ozellikler)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [8]:
#boş değerlerin eksik değerlerini boş metinlere değiştirme
for ozellik in secilen_ozellikler:
  film_verileri[ozellik]=film_verileri[ozellik].fillna("")

In [9]:
# seçilen tüm dosya özelliklerini birleştirmek.
ozellik_birlestir=film_verileri["genres"]+ " " +film_verileri["keywords"]+ " "+ film_verileri["tagline"]+ " "+film_verileri["cast"]+ " "+film_verileri["director"]
print(ozellik_birlestir)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [10]:
#Bu verileri sayısal verilere çevirme. metin verilerini özelik vektörlerine çevirme.
#sayısal veriye çevirirsek kosinüs benzerliğinden metin yakınlığını bulabiliriz.
vectorizer=TfidfVectorizer()


In [11]:
feature_vector=vectorizer.fit_transform(ozellik_birlestir)

In [12]:
print(feature_vector)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

Kosinüs benzerliğini kullanma

In [13]:
#kosinüs benzerliği kullanarak benzerlik kuvvetini  elde etmek
similarity=cosine_similarity(feature_vector)


In [14]:
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [15]:
print(similarity.shape)
#birinci index film indexi, ikincisi benzerlik değeri.

(4803, 4803)


In [16]:
# kullanııdan en sevdiği film girdisini istemek.
film_ismi=input(" En  sevdiğiniz film ismini giriniz: ")

 En  sevdiğiniz film ismini giriniz: iron man


In [14]:
# tüm filmlerin olduğu listeyi oluşturmak.
tum_title_listesi=film_verileri['title'].tolist()
print(tum_title_listesi)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [15]:
#kullanıcıdan verilen metin için yakın eşleştirmeyi bulmak
yakin_degerleri_bul=difflib.get_close_matches(film_ismi,tum_title_listesi)
print(yakin_degerleri_bul)

['Avatar']


In [16]:
en_yakin_deger=yakin_degerleri_bul[0]
print(en_yakin_deger)

Avatar


In [17]:
# title ile filmin indexini bulmak
filmlerin_indexi=film_verileri[film_verileri.title==en_yakin_deger]['index'].values[0]
print(filmlerin_indexi)

0


In [18]:
#benzer filmlerin listesine ulaşma
yakinlik_degeri=list(enumerate(similarity[filmlerin_indexi]))
print(yakinlik_degeri)

[(0, 1.0000000000000002), (1, 0.07219486822992488), (2, 0.03773299957717928), (3, 0.012520204623868901), (4, 0.10702574467235307), (5, 0.07786899789424222), (6, 0.008237143013608844), (7, 0.03613473061484884), (8, 0.02960930964063025), (9, 0.026287167439951735), (10, 0.09261074046755371), (11, 0.012717759249124133), (12, 0.027217360083100113), (13, 0.029569752523347505), (14, 0.06915925473724743), (15, 0.01955159449930903), (16, 0.034263405780616416), (17, 0.02603656461429414), (18, 0.05714759266672412), (19, 0.0389505953521203), (20, 0.039714802154154946), (21, 0.012014803805656128), (22, 0.03043869426198959), (23, 0.04592653558817949), (24, 0.046239890179652576), (25, 0.042849260959502256), (26, 0.07010711150614285), (27, 0.03719807623232885), (28, 0.04083909796927843), (29, 0.03858648330156397), (30, 0.07893753610792024), (31, 0.06055522138055148), (32, 0.030362745635800832), (33, 0.035910214700688683), (34, 0.0), (35, 0.03769674103474844), (36, 0.04891087950911423), (37, 0.08575173

In [19]:
len(yakinlik_degeri)

4803

In [20]:
# filmleri benzerliklerine göre sıralamak.azalan düzene göre sıralamak
benzer_filmleri_sirala=sorted(yakinlik_degeri,key= lambda x:x[1],reverse=True)
print(benzer_filmleri_sirala)

[(0, 1.0000000000000002), (3158, 0.24946766307532411), (2403, 0.24841462595906275), (94, 0.24505931974059814), (56, 0.2037806964828543), (47, 0.20115287461144912), (1053, 0.19702752258651426), (838, 0.18017023369312357), (3730, 0.17646241185313413), (4593, 0.1744884579741517), (239, 0.17441748680810654), (1531, 0.1682605817219649), (2696, 0.16503460259176522), (812, 0.16062301907491788), (643, 0.15644455512484967), (4401, 0.15468923545220403), (2198, 0.15217161971893783), (770, 0.1502572672775349), (1951, 0.14933372705282924), (2229, 0.14661801285492257), (1922, 0.1448197430191312), (206, 0.14226144606175542), (3208, 0.1401230206493547), (1759, 0.13899056016968867), (43, 0.1348209130228474), (1473, 0.13476547670086914), (278, 0.13291021545504), (158, 0.1325289213162767), (1650, 0.13024318650645417), (1275, 0.12602216304791153), (3439, 0.12480340331169382), (661, 0.12153002734138185), (3202, 0.12144749322246051), (4332, 0.12002556168548507), (3105, 0.11948466494212534), (775, 0.11847062

In [21]:
# benzer filmlerin isimlerini yazdırma indexine göre
print(" Sizin için önerilen filmler şunlar: \n")
i=1
for film in benzer_filmleri_sirala:
  index=film[0]
  title_from_index=film_verileri[film_verileri.index==index]["title"].values[0]
  if(i<30):
    print(i,".",title_from_index)
    i+=1


 Sizin için önerilen filmler şunlar: 

1 . Avatar
2 . Alien
3 . Aliens
4 . Guardians of the Galaxy
5 . Star Trek Beyond
6 . Star Trek Into Darkness
7 . Galaxy Quest
8 . Alien³
9 . Cargo
10 . Trekkies
11 . Gravity
12 . Moonraker
13 . Jason X
14 . Pocahontas
15 . Space Cowboys
16 . The Helix... Loaded
17 . Lockout
18 . Event Horizon
19 . Space Dogs
20 . Machete Kills
21 . Gettysburg
22 . Clash of the Titans
23 . Star Wars: Clone Wars: Volume 1
24 . The Right Stuff
25 . Terminator Salvation
26 . The Astronaut's Wife
27 . Planet of the Apes
28 . Star Trek
29 . Wing Commander


Film öneri sistemimizin derlenmiş son hali bu şekildedir:


In [19]:
film_ismi = input(' En sevdiğiniz yabancı filmi giriniz : ')

tum_title_listesi = film_verileri['title'].tolist()

yakin_degerleri_bul = difflib.get_close_matches(film_ismi, tum_title_listesi)

en_yakin_deger = yakin_degerleri_bul[0]

filmlerin_indexi = film_verileri[film_verileri.title == en_yakin_deger]['index'].values[0]

yakinlik_degeri = list(enumerate(similarity[filmlerin_indexi]))

benzer_filmleri_sirala = sorted(yakinlik_degeri, key = lambda x:x[1], reverse = True)

print('Sizin için seçilen filmler sıralanmıştır. Keyifli Seyirler : \n')

i = 1
print(f"Yazdığınız metine göre en yakın film ismi {en_yakin_deger}' tir")
for film in benzer_filmleri_sirala:
  index=film[0]
  title_from_index=film_verileri[film_verileri.index==index]["title"].values[0]
  if(i<30):
    print(i,".",title_from_index)
    i+=1

 En sevdiğiniz yabancı filmi giriniz : blader
Sizin için seçilen filmler sıralanmıştır. Keyifli Seyirler : 

Yazdığınız metine göre en yakın film ismi Blade' tir
1 . Blade
2 . Blade: Trinity
3 . Blade II
4 . The League of Extraordinary Gentlemen
5 . Eddie: The Sleepwalking Cannibal
6 . Priest
7 . Underworld: Awakening
8 . Vampire in Brooklyn
9 . The Twilight Saga: Eclipse
10 . The Lost Boys
11 . BloodRayne
12 . Dracula
13 . FearDotCom
14 . Underworld
15 . Dracula 2000
16 . Abraham Lincoln: Vampire Hunter
17 . Dracula Untold
18 . Sugar Hill
19 . Let Me In
20 . Ghost Rider
21 . Mo' Better Blues
22 . American Hero
23 . First Love, Last Rites
24 . Secret Window
25 . The Lego Movie
26 . The Wendell Baker Story
27 . Somewhere
28 . Undisputed
29 . Reindeer Games


Bu kodu çalıştırınca şu sorulacak: bir film girin
bunun sonucunda tavsiyeler vereccek